In [75]:
from chronos import BaseChronosPipeline, Chronos2Pipeline
import pandas as pd
import os
from dotenv import load_dotenv
import requests
from utils import upload
import dropbox
from pandas.tseries.frequencies import to_offset
import holidays
import torch

load_dotenv()

True

In [76]:
try:
    chronos_gpu_or_cpu = os.getenv("CHRONOS_GPU_OR_CPU", "gpu").lower()
    if chronos_gpu_or_cpu not in ["gpu", "cpu"]:
        raise ValueError("Invalid value for CHRONOS_GPU_OR_CPU. Must be 'gpu' or 'cpu'.")

    if chronos_gpu_or_cpu == "gpu":


        torch.cuda.set_device(int(os.getenv("CHRONOS_GPU_DEVICE", 0)))

        # Load the Chronos-2 pipeline
        # GPU recommended for faster inference, but CPU is also supported
        pipeline: Chronos2Pipeline = BaseChronosPipeline.from_pretrained(
            "amazon/chronos-2",
            device_map="cuda"
            # device_map="cpu"
    )
    else:
        
        # Load the Chronos-2 pipeline
        # GPU recommended for faster inference, but CPU is also supported
        pipeline: Chronos2Pipeline = BaseChronosPipeline.from_pretrained(
            "amazon/chronos-2",
            device_map="cpu"
    )
except Exception as e:
    print(f"Error loading Chronos-2 pipeline: {e}")
    
    pipeline: Chronos2Pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-2",
    device_map="cpu")
    raise


In [77]:
def regularize_hourly(g: pd.DataFrame) -> pd.DataFrame:
    """
    Reindex each group's timestamps to strict hourly and fill gaps.
    Works whether the grouping column is present or omitted (include_groups=False).
    """
    # The group key (id) is available as g.name; if ID_COL exists, prefer it.
    sid = g[ID_COL].iloc[0] if ID_COL in g.columns else g.name

    g = g.sort_values(TS_COL)
    full_idx = pd.date_range(g[TS_COL].min(), g[TS_COL].max(), freq="h")
    g = g.set_index(TS_COL).reindex(full_idx)
    g.index.name = TS_COL

    # restore id (constant for the whole group)
    g[ID_COL] = sid

    # numeric + fill for targets
    for col in TARGETS:
        if col in g.columns:
            g[col] = pd.to_numeric(g[col], errors="coerce").ffill().bfill()
    return g.reset_index()

def add_holiday_flags(
    df: pd.DataFrame,
    ts_col: str = "ds",
    local_tz: str = "America/Montreal",
    observed: bool = True,
    include_names: bool = False,
) -> pd.DataFrame:
    """
    Adds boolean columns:
      • is_qc_holiday       — Québec public holiday (CA-QC)
      • is_jewish_holiday   — Israeli public/Jewish holiday (IL)
    Optionally adds:
      • qc_holiday_name
      • jewish_holiday_name

    Notes:
      • Holiday checks are date-based (00:00–24:00 local calendar date),
        not sundown-to-sundown observance.
      • NaT timestamps are ignored gracefully.
    """
    out = df.copy()

    # 1) Parse to datetime
    out[ts_col] = pd.to_datetime(out[ts_col], errors="coerce")

    # 2) Get the calendar DATE to use for holiday lookup
    #    - If tz-aware: convert to Montreal then take .date
    #    - If naive: assume values already represent local Montreal wall-clock; just take .date
    if getattr(out[ts_col].dt, "tz", None) is not None:
        dates_for_calendar = out[ts_col].dt.tz_convert(local_tz).dt.date
    else:
        dates_for_calendar = out[ts_col].dt.date

    # 3) Build a SAFE integer year range for the holiday objects
    years_series = pd.Series(dates_for_calendar)
    years_series = years_series.dropna().map(lambda d: int(pd.Timestamp(d).year))
    if years_series.empty:
        raise ValueError("No valid datetimes found to extract holiday years.")
    years = list(range(int(years_series.min()), int(years_series.max()) + 1))

    # 4) Construct holiday calendars
    qc_holidays = holidays.Canada(subdiv="QC", years=years, observed=observed)
    il_holidays = holidays.Israel(years=years, observed=observed)

   # 5) Flag membership
    out["is_qc_holiday"] = [ ("yes" if d in qc_holidays else "no") if pd.notna(pd.Timestamp(d)) else "no"
                             for d in dates_for_calendar ]
    out["is_jewish_holiday"] = [ ("yes" if d in il_holidays else "no") if pd.notna(pd.Timestamp(d)) else "no"
                                 for d in dates_for_calendar ]

    if include_names:
        out["qc_holiday_name"] = [ qc_holidays.get(d, "no") if pd.notna(pd.Timestamp(d)) else "no"
                                   for d in dates_for_calendar ]
        out["jewish_holiday_name"] = [ il_holidays.get(d, "no") if pd.notna(pd.Timestamp(d)) else "no"
                                       for d in dates_for_calendar ]

    return out

shift_types_dict = {'W1':'flow',
 'X1':'pod',
 'X3':'pod',
 'X4':'vertical',
 'X2':'vertical',
 'WOC1':'oncall',
 'WOC2':'oncall',
 'WOC3':'oncall',
 'X5':'pod',
 'W3':'overlap',
 'Y1':'pod',
 'Y3':'pod',
 'Y4':'vertical',
 'Y2':'vertical',
 'Y5':'pod',
 'Z1':'night',
 'Z2':'night',
 'D1':'pod',
 'R1':'pod',
 'P1':'vertical',
 'D2':'vertical',
 'OC1':'oncall',
 'OC2':'oncall',
 'V1':'flow',
 'A1':'pod',
 'G1':'vertical',
 'E1':'pod',
 'R2':'pod',
 'A2':'pod',
 'P2':'vertical',
 'E2':'vertical',
 'N1':'night',
 'N2':'night',
 'L2':'overlap',
 'L4':'overlap',
 'H1':'teaching',
 'B1':'vertical',
 'L1':'overlap',
 'W5':'overlap',
 'L6':'overlap',
 'B2':'vertical'}

In [78]:
# Load hourly data
df = pd.read_csv(
    'https://www.dropbox.com/scl/fi/s83jig4zews1xz7vhezui/allDataWithCalculatedColumns.csv?rlkey=9mm4zwaugxyj2r4ooyd39y4nl&raw=1')
df.ds = pd.to_datetime(df.ds, errors="coerce")
df['id'] = 'jgh'

hourly_report_df = df.copy()
df.tail()

,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,RAZ_IMCONS_MORE4H,RAZ_XRAY_MORE2H,RAZ_CT_MORE2H1,PSYCH1,PSYCH_WAITINGADM,total_tbs,vert_tbs,pod_tbs,overflow,id
44873,2026-02-18 16:00:00,13,99,5,69,18,168,0,24,0,...,0,4,4,13,8,38,31,7,13,jgh
44874,2026-02-18 17:00:00,5,104,3,72,8,176,0,24,0,...,0,4,3,12,8,31,27,4,12,jgh
44875,2026-02-18 18:00:00,9,113,3,75,12,188,1,25,0,...,0,1,1,11,8,35,32,3,9,jgh
44876,2026-02-18 19:00:00,14,127,2,77,16,204,5,30,0,...,0,0,3,9,8,32,24,8,13,jgh
44877,2026-02-18 20:00:00,7,134,3,80,10,214,3,33,0,...,0,0,1,9,7,34,21,13,13,jgh


In [79]:
# Load shift data
all_shifts_df = pd.read_csv('https://www.dropbox.com/scl/fi/yeyr2a7pj6nry8i2q3m0c/all_shifts.csv?rlkey=q1su2h8fqxfnlu7t1l2qe1w0q&raw=1')
all_shifts_df['shift_start'] = pd.to_datetime(all_shifts_df['shift_start']).dt.round('h')
all_shifts_df['shift_end'] = pd.to_datetime(all_shifts_df['shift_end']).dt.round('h')
all_shifts_df['shift_type'] = all_shifts_df['shift_short_name'].map(shift_types_dict)
all_shifts_df.tail()

,scheduled_shift_id,group_id,user_id,employee_id,npi,first_name,last_name,facility_id,facility_ext_id,facility_name,...,shift_start,shift_end,shift_hours,work_start,work_end,work_hours,count_as_shift,is_night,is_weekend,shift_type
29996,88886,1,28,NaN,NaN,Emily,Moras,1,NaN,Jewish General Hospital,...,2026-02-26 16:00:00,2026-02-27 00:00:00,8.00,2/26/2026 16:00,2/27/2026 00:00,8.00,1,0,0,vertical
29997,88884,1,73,NaN,NaN,Madison,Meehan,1,NaN,Jewish General Hospital,...,2026-02-26 16:00:00,2026-02-27 00:00:00,8.00,2/26/2026 16:00,2/27/2026 00:00,8.00,1,0,0,pod
29998,88887,1,80,NaN,NaN,Natasha,Poirier,1,NaN,Jewish General Hospital,...,2026-02-26 18:00:00,2026-02-27 02:00:00,8.00,2/26/2026 18:00,2/27/2026 02:00,8.00,1,0,0,vertical
29999,88219,1,12,NaN,NaN,Julia,Bernard,1,NaN,Jewish General Hospital,...,2026-02-27 00:00:00,2026-02-27 08:00:00,8.08,2/26/2026 23:45,2/27/2026 07:50,8.08,1,1,0,night
30000,88888,1,61,NaN,NaN,Alexander,Hart,1,NaN,Jewish General Hospital,...,2026-02-27 00:00:00,2026-02-27 08:00:00,8.08,2/26/2026 23:45,2/27/2026 07:50,8.08,1,1,0,night


In [80]:
# Create hourly rows
# We'll use a list comprehension to generate the range for each row
expanded_rows = []
for _, row in all_shifts_df.iterrows():
    # Create range. inclusive='left' means [start, end)
    # If start == end (e.g. 0 length shift after rounding), it will be empty, which is correct
    hours = pd.date_range(row['shift_start'], row['shift_end'], freq='h', inclusive='left')
    for h in hours:
        expanded_rows.append({
            'ds': h,
            'user': row['first_name']+row['last_name'],
            'shift_type': row['shift_type'],
            'shift_short_name': row['shift_short_name']
        })

expanded_df = pd.DataFrame(expanded_rows)

expanded_df.tail()

,ds,user,shift_type,shift_short_name
273246,2026-02-27 03:00:00,AlexanderHart,night,N2
273247,2026-02-27 04:00:00,AlexanderHart,night,N2
273248,2026-02-27 05:00:00,AlexanderHart,night,N2
273249,2026-02-27 06:00:00,AlexanderHart,night,N2
273250,2026-02-27 07:00:00,AlexanderHart,night,N2


In [81]:
# Pivot
# index=timestamp, columns=user_id, values=shift_type
hourly_shifts_by_user_df = expanded_df.pivot_table(
    index='ds', 
    columns='user', 
    values='shift_type', 
    aggfunc='first' # In case of duplicates, take the first
)

# Fill NaNs
hourly_shifts_by_user_df = hourly_shifts_by_user_df.fillna('NotWorking')
hourly_shifts_by_user_df.tail()

user,AlanAzuelos,AlexGuttman,AlexanderHart,AlexandreDostaler,AlexisHaligua,AmelieBellemare,ArzuChaudhry,BernardUnger,DahliaGuttman,DanielMankarios,...,PaulBrisebois,PhilipStasiak,RafaelAroutiunian,SaraAhronheim,ShuoPeng,StephenRosenthal,TanLe,ThuHangTran,WayneChoi,WillGrad
ds,,,,,,,,,,,,,,,,,,,,,
2026-02-27 03:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
2026-02-27 04:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
2026-02-27 05:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
2026-02-27 06:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
2026-02-27 07:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking


In [82]:
df.columns.tolist()

['ds',
 'INFLOW_STRETCHER',
 'Infl_Stretcher_cum',
 'INFLOW_AMBULATORY',
 'Infl_Ambulatory_cum',
 'Inflow_Total',
 'Inflow_Cum_Total',
 'INFLOW_AMBULANCES',
 'Infl_Ambulances_cum',
 'FLS',
 'CUM_ADMREQ',
 'CUM_BA1',
 'WAITINGADM',
 'TTStr',
 'TRG_HALLWAY1',
 'TRG_HALLWAY_TBS',
 'reoriented_cum',
 'reoriented_cum_MD',
 'QTRACK1',
 'RESUS',
 'Pod_T',
 'POD_GREEN',
 'POD_GREEN_TBS',
 'POD_YELLOW',
 'POD_YELLOW_TBS',
 'POD_ORANGE',
 'POD_ORANGE_TBS',
 'POD_CONS_MORE2H',
 'POD_IMCONS_MORE4H',
 'POD_XRAY_MORE2H',
 'POD_CT_MORE2H',
 'POST_POD1',
 'VERTSTRET',
 'RAZ_TBS',
 'RAZ_LAZYBOY',
 'RAZ_WAITINGREZ',
 'AMBVERT1',
 'AMBVERTTBS',
 'QTrack_TBS',
 'Garage_TBS',
 'RAZ_CONS_MORE2H',
 'RAZ_IMCONS_MORE4H',
 'RAZ_XRAY_MORE2H',
 'RAZ_CT_MORE2H1',
 'PSYCH1',
 'PSYCH_WAITINGADM',
 'total_tbs',
 'vert_tbs',
 'pod_tbs',
 'overflow',
 'id']

In [83]:
ID_COL = "id"
TS_COL = "ds"
# TARGETS = ['total_tbs', 'Inflow_Total', 'overflow']
# Targets are all columns in df except ds (timestamp) and id
TARGETS = [col for col in df.columns.tolist() if col != TS_COL and col != ID_COL]

df = df.copy()
df[TS_COL] = pd.to_datetime(df[TS_COL], errors="coerce")
df = df.dropna(subset=[TS_COL])

# Snap to exact hours (lowercase 'h' to avoid FutureWarning)
df[TS_COL] = df[TS_COL].dt.floor("h")

# Sort + dedupe
df = df.sort_values([ID_COL, TS_COL]).drop_duplicates(
    [ID_COL, TS_COL], keep="last")





# Call apply with include_groups=False if supported; else fall back
gb = df.groupby(ID_COL, group_keys=False)
try:
    df = gb.apply(regularize_hourly, include_groups=False)
except TypeError:
    # older pandas without include_groups
    df = gb.apply(regularize_hourly)

# Assert truly hourly (accept 'h' and 'H')
g = df[df[ID_COL] == "jgh"].sort_values(TS_COL)
freq = pd.infer_freq(g[TS_COL])
if not freq:
    raise ValueError("No inferable frequency after regularization.")
if to_offset(freq).name.lower() != "h":
    # extra check independent of infer_freq
    diffs = g[TS_COL].diff().dropna()
    bad = g.loc[diffs != pd.Timedelta(hours=1), TS_COL].head(10).tolist()
    raise ValueError(f"Non-1h gaps remain around: {bad}")

In [84]:
# Predict
print('Predicting basic forecast')
basic_forecast = pipeline.predict_df(
    df,
    prediction_length=24,
    # future_df = future_df.head(24),
    # quantile_levels=[0.1, 0.5, 0.9],
    # quantile_levels=[0.5],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)

basic_forecast.head()

Predicting basic forecast


,id,ds,target_name,predictions,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,jgh,2026-02-18 21:00:00,INFLOW_STRETCHER,5.933879,3.241724,4.114210,4.755703,5.348392,5.933879,6.546845,7.191843,8.025124,9.271070
1,jgh,2026-02-18 22:00:00,INFLOW_STRETCHER,5.025776,2.482104,3.274783,3.883570,4.452264,5.025776,5.635357,6.281515,7.101665,8.303295
2,jgh,2026-02-18 23:00:00,INFLOW_STRETCHER,4.284609,1.926104,2.622321,3.177419,3.716316,4.284609,4.886617,5.519050,6.325222,7.515788
3,jgh,2026-02-19 00:00:00,INFLOW_STRETCHER,3.574238,1.497822,2.072262,2.560669,3.055437,3.574238,4.118187,4.706468,5.471491,6.579696
4,jgh,2026-02-19 01:00:00,INFLOW_STRETCHER,2.959300,1.242124,1.755014,2.165630,2.546544,2.959300,3.443124,3.969638,4.655984,5.673961


In [85]:
basic_forecast.target_name.value_counts()

target_name
INFLOW_STRETCHER       24
Infl_Stretcher_cum     24
INFLOW_AMBULATORY      24
Infl_Ambulatory_cum    24
Inflow_Total           24
Inflow_Cum_Total       24
INFLOW_AMBULANCES      24
Infl_Ambulances_cum    24
FLS                    24
CUM_ADMREQ             24
CUM_BA1                24
WAITINGADM             24
TTStr                  24
TRG_HALLWAY1           24
TRG_HALLWAY_TBS        24
reoriented_cum         24
reoriented_cum_MD      24
QTRACK1                24
RESUS                  24
Pod_T                  24
POD_GREEN              24
POD_GREEN_TBS          24
POD_YELLOW             24
POD_YELLOW_TBS         24
POD_ORANGE             24
POD_ORANGE_TBS         24
POD_CONS_MORE2H        24
POD_IMCONS_MORE4H      24
POD_XRAY_MORE2H        24
POD_CT_MORE2H          24
POST_POD1              24
VERTSTRET              24
RAZ_TBS                24
RAZ_LAZYBOY            24
RAZ_WAITINGREZ         24
AMBVERT1               24
AMBVERTTBS             24
QTrack_TBS             24


In [86]:
# basic_forecast


df_with_holidays = add_holiday_flags(df, ts_col='ds', include_names=True)

#create a dataframe with the next 24 hours timestamps hourly as column 'ds', with column 'id' jgh
future_df = hourly_shifts_by_user_df.reset_index()[hourly_shifts_by_user_df.reset_index()['ds'] > df['ds'].max()]
future_df['id'] = 'jgh'
future_df = add_holiday_flags(future_df, ts_col='ds', include_names=True)

# First, add holiday flags to future_df
future_df_with_added_holidays = add_holiday_flags(future_df, ts_col='ds', include_names=True)

# Then, select only the columns from future_df_with_added_holidays that are also in df_with_holidays
common_columns = [col for col in future_df_with_added_holidays.columns if col in df_with_holidays.columns]
future_df_with_holidays = future_df_with_added_holidays[common_columns]

# Predict
print('Predicting forecast with holidays')  
forecast_with_holidays = pipeline.predict_df(
    df_with_holidays,
    prediction_length=24,
    future_df = future_df_with_holidays.head(24),
    # quantile_levels=[0.1, 0.5, 0.9],
    # quantile_levels=[0.5],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)
forecast_with_holidays.tail()

Predicting forecast with holidays


,id,ds,target_name,predictions,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
1171,jgh,2026-02-19 16:00:00,overflow,14.241021,6.647802,9.108795,10.984147,12.625793,14.241021,15.999485,17.787617,20.071587,23.267504
1172,jgh,2026-02-19 17:00:00,overflow,13.177074,5.993545,8.128353,9.879399,11.538149,13.177074,14.916600,16.721312,19.044037,22.295208
1173,jgh,2026-02-19 18:00:00,overflow,11.841656,5.352747,7.374962,8.965891,10.407210,11.841656,13.408037,15.079602,17.285894,20.550558
1174,jgh,2026-02-19 19:00:00,overflow,10.914460,4.922644,6.823212,8.283416,9.593200,10.914460,12.371221,13.951403,15.983454,18.908556
1175,jgh,2026-02-19 20:00:00,overflow,9.870110,4.482102,6.146929,7.447190,8.648263,9.870110,11.208641,12.650315,14.524948,17.272247


In [87]:
df_with_staffing = df.merge(hourly_shifts_by_user_df, on='ds')
future_df_with_staffing = hourly_shifts_by_user_df.reset_index()[hourly_shifts_by_user_df.reset_index()['ds'] > df['ds'].max()]
future_df_with_staffing['id'] = 'jgh'

print('Predicting forecast with staffing')
forecast_with_staffing = pipeline.predict_df(
    df_with_staffing,
    prediction_length=24,
    future_df = future_df_with_staffing.head(24),
    # quantile_levels=[0.1, 0.5, 0.9],
    # quantile_levels=[0.5],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)

forecast_with_staffing.tail()

Predicting forecast with staffing


,id,ds,target_name,predictions,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
1171,jgh,2026-02-19 16:00:00,overflow,13.691640,5.578275,8.041115,10.080638,11.948416,13.691640,15.567362,17.458374,19.825645,23.094517
1172,jgh,2026-02-19 17:00:00,overflow,12.783810,4.782696,6.951960,8.976368,10.915960,12.783810,14.717997,16.717449,19.252533,22.670820
1173,jgh,2026-02-19 18:00:00,overflow,11.583679,4.202866,6.438557,8.358477,10.044557,11.583679,13.362259,15.306893,17.849674,21.453724
1174,jgh,2026-02-19 19:00:00,overflow,10.890186,4.235586,6.274891,7.930351,9.445219,10.890186,12.475585,14.229650,16.489841,19.604252
1175,jgh,2026-02-19 20:00:00,overflow,9.881523,4.110954,5.911648,7.296070,8.574196,9.881523,11.326986,12.961377,15.137604,18.114174


In [88]:
weather_df = pd.read_csv('https://www.dropbox.com/scl/fi/gmhwwld9z9yychg4r0yuk/weather.csv?rlkey=66c78m90aviamr0x0uu72pfr8&raw=1')
weather_df.ds = pd.to_datetime(weather_df.ds, errors="coerce")


future_weather_df = weather_df[weather_df.ds > df.ds.max()].head(24)
future_weather_df['id']='jgh'

print('Predicting forecast with weather')
# Predict
forecast_with_weather = pipeline.predict_df(
    #join df with weather_df on ds
    df.merge(weather_df, on='ds'),
    prediction_length=24,
    #weather_df where ds is greater than the max of df.ds.max()
    future_df = future_weather_df,
    # future_df = future_df.head(24),
    # quantile_levels=[0.1, 0.5, 0.9],
    quantile_levels=[0.5],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)

Predicting forecast with weather


In [89]:
weather_df.tail()

,ds,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,wind_speed_10m,wind_direction_10m,wind_gusts_10m,precipitation_probability
45144,2026-02-25 00:00:00,-11.460999,82.0,-13.919160,-16.927895,0.0,0.0,0.0,0.29,2.0,1016.400024,986.932068,71.0,62.0,5.0,25.0,13.324863,268.451874,36.360001,2.0
45145,2026-02-25 01:00:00,-12.210999,83.0,-14.505652,-17.501385,0.0,0.0,0.0,0.29,2.0,1017.099976,987.528625,74.0,63.0,27.0,21.0,11.879999,270.000000,34.200001,4.0
45146,2026-02-25 02:00:00,-12.710999,84.0,-14.850284,-17.864195,0.0,0.0,0.0,0.28,3.0,1017.599976,987.958069,83.0,71.0,57.0,12.0,10.805999,271.909119,30.599998,4.0
45147,2026-02-25 03:00:00,-13.261000,85.0,-15.246855,-18.226086,0.0,0.0,0.0,0.28,3.0,1018.099976,988.381836,90.0,77.0,81.0,5.0,9.366920,272.202545,25.919998,4.0
45148,2026-02-25 04:00:00,-14.011000,86.0,-15.843117,-18.692905,0.0,0.0,0.0,0.28,3.0,1018.500000,988.685791,94.0,80.0,95.0,2.0,7.208994,272.862335,18.719999,4.0


In [90]:
# All variables forecast
print('Predicting all variables forecast')
all_variable_df = add_holiday_flags(df_with_staffing, ts_col='ds', include_names=True).merge(weather_df, on='ds')

forecast_all_vars_with_future = pipeline.predict_df(
    all_variable_df,
    prediction_length=24,
    #future_df should be future_df_with_staffing merged with future_weather_df on 'ds' and 'id'
    future_df = future_df_with_staffing.merge(future_weather_df, on=['ds', 'id']),
    # quantile_levels=[0.1, 0.5, 0.9],
    quantile_levels=[0.2,0.5,0.8],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)   
forecast_all_vars_with_future.head()

Predicting all variables forecast


,id,ds,target_name,predictions,0.2,0.5,0.8
0,jgh,2026-02-18 21:00:00,INFLOW_STRETCHER,5.779692,3.952271,5.779692,7.914401
1,jgh,2026-02-18 22:00:00,INFLOW_STRETCHER,4.921159,3.175433,4.921159,6.980110
2,jgh,2026-02-18 23:00:00,INFLOW_STRETCHER,4.224596,2.591596,4.224596,6.212049
3,jgh,2026-02-19 00:00:00,INFLOW_STRETCHER,3.502632,2.031728,3.502632,5.310541
4,jgh,2026-02-19 01:00:00,INFLOW_STRETCHER,2.951257,1.720697,2.951257,4.622687


In [91]:
#join the predictions columns of basic_forecast, forecast_with_holidays, forecast_with_staffing, forecast_with_weather, forecast_all_vars_without_future, forecast_all_vars_with_future on the 'ds' column
basic_forecast = basic_forecast[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'basic_forecast'})
forecast_with_holidays = forecast_with_holidays[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'forecast_with_holidays'})
forecast_with_staffing = forecast_with_staffing[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'forecast_with_staffing'})
forecast_with_weather = forecast_with_weather[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'forecast_with_weather'})
# forecast_all_vars_without_future = forecast_all_vars_without_future[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'forecast_all_vars_without_future'})
forecast_all_vars_with_future_to_merge = forecast_all_vars_with_future[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'forecast_all_vars_with_future'})

pred_df = basic_forecast.merge(forecast_with_holidays, on=['ds', 'target_name']).merge(forecast_with_staffing, on=['ds', 'target_name']).merge(forecast_with_weather, on=['ds', 'target_name']).merge(forecast_all_vars_with_future_to_merge, on=['ds', 'target_name'])
pred_df.head()

,ds,target_name,basic_forecast,forecast_with_holidays,forecast_with_staffing,forecast_with_weather,forecast_all_vars_with_future
0,2026-02-18 21:00:00,INFLOW_STRETCHER,5.933879,5.919996,5.882460,5.815547,5.779692
1,2026-02-18 22:00:00,INFLOW_STRETCHER,5.025776,5.007184,4.946616,4.986432,4.921159
2,2026-02-18 23:00:00,INFLOW_STRETCHER,4.284609,4.270706,4.282044,4.262690,4.224596
3,2026-02-19 00:00:00,INFLOW_STRETCHER,3.574238,3.588573,3.699302,3.466273,3.502632
4,2026-02-19 01:00:00,INFLOW_STRETCHER,2.959300,2.962415,3.121028,2.871081,2.951257


In [92]:
# Create a new dataframe with the average % difference of each forecast compared to the basic forecast, for each target_name averaged over all ds
# The output dataframe should only have 1 row per target_name, and columns for the average % difference of each forecast compared to the basic forecast
comparison_df = pred_df.copy()
comparison_df['%diff_holidays'] = (comparison_df['forecast_with_holidays'] - comparison_df['basic_forecast']) / comparison_df['basic_forecast'] * 100
comparison_df['%diff_staffing'] = (comparison_df['forecast_with_staffing'] - comparison_df['basic_forecast']) / comparison_df['basic_forecast'] * 100
comparison_df['%diff_weather'] = (comparison_df['forecast_with_weather'] - comparison_df['basic_forecast']) / comparison_df['basic_forecast'] * 100
comparison_df['%diff_all_vars_with_future'] = (comparison_df['forecast_all_vars_with_future'] - comparison_df['basic_forecast']) / comparison_df['basic_forecast'] * 100
comparison_df = comparison_df.groupby('target_name')[['%diff_holidays', '%diff_staffing', '%diff_weather', '%diff_all_vars_with_future']].mean().reset_index()
comparison_df.head()
# comparison_df.to_csv('forecast_variable_effects.csv', index=False)

,target_name,%diff_holidays,%diff_staffing,%diff_weather,%diff_all_vars_with_future
0,AMBVERT1,-0.205424,8.447520,5.038006,11.032767
1,AMBVERTTBS,-0.121161,10.019727,4.164478,10.933830
2,CUM_ADMREQ,-0.279040,-0.247097,-2.859148,-1.448173
3,CUM_BA1,-0.112012,1.463895,1.921420,4.003266
4,FLS,53.232315,1316.664917,252.826843,1590.374023


In [93]:
anomaly_detection_ranges_df = pd.read_csv('https://www.dropbox.com/scl/fi/fjz0am427gw35sz7l994m/anomaly_detection_ranges.csv?rlkey=lib9w0jz2zei5n566jv76o7ol&raw=1')
anomaly_detection_ranges_df.ds = pd.to_datetime(anomaly_detection_ranges_df.ds, errors="coerce")
anomaly_detection_ranges_df.tail()

,ds,INFLOW_STRETCHER_yhat,INFLOW_STRETCHER_yhat_lower,INFLOW_STRETCHER_yhat_upper,Infl_Stretcher_cum_yhat,Infl_Stretcher_cum_yhat_lower,Infl_Stretcher_cum_yhat_upper,INFLOW_AMBULATORY_yhat,INFLOW_AMBULATORY_yhat_lower,INFLOW_AMBULATORY_yhat_upper,...,total_tbs_yhat_upper,vert_tbs_yhat,vert_tbs_yhat_lower,vert_tbs_yhat_upper,pod_tbs_yhat,pod_tbs_yhat_lower,pod_tbs_yhat_upper,overflow_yhat,overflow_yhat_lower,overflow_yhat_upper
331,2026-02-19 21:00:00,6.696714,1.802424,11.619308,139.707146,101.901821,177.656871,4.352476,-0.203400,8.953634,...,41.994893,18.867586,3.846503,34.279388,5.744107,0.634606,10.332182,11.145015,2.151968,19.514478
332,2026-02-19 22:00:00,6.121090,0.945304,10.982407,150.280560,115.100569,186.773367,3.820164,-1.089628,8.876808,...,38.589973,17.573136,2.665570,31.795491,5.644985,0.685674,10.533595,10.749878,2.870187,19.433805
333,2026-02-19 23:00:00,5.286725,0.190287,9.803927,139.332934,102.146741,176.226473,3.074107,-1.897070,7.819009,...,38.648552,16.934022,3.057174,32.153232,5.390424,0.502800,10.311826,10.425133,2.793916,18.719313
334,2026-02-20 00:00:00,4.413434,-0.472789,9.255871,103.442195,69.143827,141.911745,2.346290,-2.773706,7.528913,...,38.043533,17.108873,3.052264,32.388837,4.910608,-0.175624,10.142952,10.125728,2.354540,18.099791
335,2026-02-20 01:00:00,3.722666,-1.637739,8.385779,56.196857,17.703664,93.141863,1.851982,-3.068016,6.753954,...,37.470679,17.550123,3.339559,33.297021,4.254566,-0.708479,9.695353,9.853741,1.596554,18.026973


In [97]:
recent_df = df.tail(24).copy()
targets = recent_df.columns.tolist()
#remove ds and id from targets
targets = [t for t in targets if t not in ['ds', 'id']]

# Merge recent_df with anomaly_detection_ranges_df on 'ds' to align the data
recent_df = recent_df.merge(anomaly_detection_ranges_df, on='ds', how='left')

for target in targets:
    recent_df[target+'_hist_anomaly'] = ((recent_df[target] < recent_df[target+'_yhat_lower']) | (recent_df[target] > recent_df[target+'_yhat_upper'])).map({True: 'yes', False: 'no'})
    recent_df[target+'_hist_colour'] = recent_df.apply(lambda row: '#D13438' if row[target+'_hist_anomaly'] == 'yes' else ('#FFB900' if row[target] > row[target+'_yhat'] else ('#107C10' if row[target] < row[target+'_yhat'] else '#000000')), axis=1)
recent_df.tail()

/tmp/ipykernel_555394/2836290958.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  recent_df[target+'_hist_anomaly'] = ((recent_df[target] < recent_df[target+'_yhat_lower']) | (recent_df[target] > recent_df[target+'_yhat_upper'])).map({True: 'yes', False: 'no'})
/tmp/ipykernel_555394/2836290958.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  recent_df[target+'_hist_colour'] = recent_df.apply(lambda row: '#D13438' if row[target+'_hist_anomaly'] == 'yes' else ('#FFB900' if row[target] > row[target+'_yhat'] else ('#107C10' 

,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,PSYCH_WAITINGADM_hist_anomaly,PSYCH_WAITINGADM_hist_colour,total_tbs_hist_anomaly,total_tbs_hist_colour,vert_tbs_hist_anomaly,vert_tbs_hist_colour,pod_tbs_hist_anomaly,pod_tbs_hist_colour,overflow_hist_anomaly,overflow_hist_colour
19,2026-02-18 16:00:00,13.0,99.0,5.0,69.0,18.0,168.0,0.0,24.0,0.0,...,no,#FFB900,no,#FFB900,no,#FFB900,no,#107C10,no,#107C10
20,2026-02-18 17:00:00,5.0,104.0,3.0,72.0,8.0,176.0,0.0,24.0,0.0,...,no,#FFB900,no,#107C10,no,#107C10,no,#107C10,no,#107C10
21,2026-02-18 18:00:00,9.0,113.0,3.0,75.0,12.0,188.0,1.0,25.0,0.0,...,no,#FFB900,no,#FFB900,no,#FFB900,no,#107C10,no,#107C10
22,2026-02-18 19:00:00,14.0,127.0,2.0,77.0,16.0,204.0,5.0,30.0,0.0,...,no,#FFB900,no,#FFB900,no,#FFB900,no,#FFB900,no,#FFB900
23,2026-02-18 20:00:00,7.0,134.0,3.0,80.0,10.0,214.0,3.0,33.0,0.0,...,no,#107C10,no,#FFB900,no,#107C10,yes,#D13438,no,#FFB900


In [99]:
targets = pred_df['target_name'].unique().tolist()
output_df = pd.DataFrame()
for target in targets:
    target_df = forecast_all_vars_with_future[forecast_all_vars_with_future['target_name'] == target][['ds', 'predictions', '0.2', '0.8']].rename(columns={'predictions': target+'_forecast', '0.2': target+'_forecast_lower', '0.8': target+'_forecast_upper'})
    # target_df = pred_df[pred_df['target_name'] == target][['ds', 'forecast_all_vars_with_future']].rename(columns={'forecast_all_vars_with_future': target+'_forecast'})
    target_df = target_df.merge(anomaly_detection_ranges_df[['ds', target+'_yhat', target+'_yhat_lower', target+'_yhat_upper']], on=['ds'], how='left')
    target_df[target+'_anomaly'] = ((target_df[target+'_forecast'] < target_df[target+'_yhat_lower']) | (target_df[target+'_forecast'] > target_df[target+'_yhat_upper'])).map({True: 'yes', False: 'no'})
    #assign a colour based on how the value compares to the yhat and the yhat_lower and yhat_upper. If it's an anomaly, colour is #D13438. If it's between yhat and yhat_upper, colour is #FFB900. If it's between yhat_lower and yhat, colour is #107C10. 
    target_df[target+'_colour'] = target_df.apply(lambda row: '#D13438' if row[target+'_anomaly'] == 'yes' else ('#FFB900' if row[target+'_forecast'] > row[target+'_yhat'] else '#107C10'), axis=1)
    if output_df.empty:
        output_df = target_df
    else:
        output_df = output_df.merge(target_df, on='ds', how='outer')

# output_df = output_df.merge(recent_df.tail(24), on='ds', how='outer')


output_df.to_csv('ED_Hourly_Forecasts_Anomalies_v1.0.csv', index=False)

output_df.head(48)
    

,ds,INFLOW_STRETCHER_forecast,INFLOW_STRETCHER_forecast_lower,INFLOW_STRETCHER_forecast_upper,INFLOW_STRETCHER_yhat,INFLOW_STRETCHER_yhat_lower,INFLOW_STRETCHER_yhat_upper,INFLOW_STRETCHER_anomaly,INFLOW_STRETCHER_colour,Infl_Stretcher_cum_forecast,...,pod_tbs_anomaly,pod_tbs_colour,overflow_forecast,overflow_forecast_lower,overflow_forecast_upper,overflow_yhat,overflow_yhat_lower,overflow_yhat_upper,overflow_anomaly,overflow_colour
0,2026-02-18 21:00:00,5.779692,3.952271,7.914401,6.626715,1.705719,11.518458,no,#107C10,136.840027,...,yes,#D13438,11.849062,10.736015,13.010250,11.503666,4.051061,19.841473,no,#FFB900
1,2026-02-18 22:00:00,4.921159,3.175433,6.980110,6.054769,1.040691,10.766856,no,#107C10,141.235413,...,no,#FFB900,10.816630,9.332843,12.359325,11.127977,2.936752,19.020845,no,#107C10
2,2026-02-18 23:00:00,4.224596,2.591596,6.212049,5.224543,0.441921,10.257535,no,#107C10,143.577530,...,no,#FFB900,10.279194,8.436895,12.140488,10.824984,2.660378,18.808131,no,#107C10
3,2026-02-19 00:00:00,3.502632,2.031728,5.310541,4.355933,-0.693936,9.374298,no,#107C10,149.065384,...,no,#FFB900,9.936988,7.884090,12.050665,10.549487,2.154142,18.250485,no,#107C10
4,2026-02-19 01:00:00,2.951257,1.720697,4.622687,3.670480,-1.600233,8.456363,no,#107C10,7.871067,...,no,#107C10,9.407598,7.247963,11.656242,10.303404,2.410038,18.380733,no,#107C10
5,2026-02-19 02:00:00,2.392483,1.427630,3.778717,3.232897,-1.732512,8.242197,no,#107C10,9.165661,...,no,#107C10,9.112713,6.984207,11.481449,10.117261,2.139609,18.729844,no,#107C10
6,2026-02-19 03:00:00,2.270979,1.323721,3.679122,2.960760,-1.766084,8.042802,no,#107C10,9.599552,...,no,#107C10,9.072615,6.882127,11.601955,10.023391,1.934722,17.353727,no,#107C10
7,2026-02-19 04:00:00,2.384673,1.362587,3.860559,2.782302,-2.085866,7.519497,no,#107C10,10.456776,...,no,#107C10,9.147820,6.878465,11.733626,10.039323,2.462977,17.729277,no,#107C10
8,2026-02-19 05:00:00,2.531496,1.435462,4.086374,2.778933,-1.925326,7.319861,no,#107C10,12.773651,...,no,#107C10,9.248342,6.836246,11.931195,10.160816,2.006067,18.108501,no,#107C10
9,2026-02-19 06:00:00,2.915952,1.679689,4.364771,3.163639,-2.004028,7.800577,no,#107C10,16.302940,...,no,#107C10,9.250228,6.775894,12.065121,10.359375,2.652605,19.079731,no,#107C10


In [100]:
output_df.merge(recent_df, on='ds', how='outer')


,ds,INFLOW_STRETCHER_forecast,INFLOW_STRETCHER_forecast_lower,INFLOW_STRETCHER_forecast_upper,INFLOW_STRETCHER_yhat_x,INFLOW_STRETCHER_yhat_lower_x,INFLOW_STRETCHER_yhat_upper_x,INFLOW_STRETCHER_anomaly,INFLOW_STRETCHER_colour,Infl_Stretcher_cum_forecast,...,PSYCH_WAITINGADM_hist_anomaly,PSYCH_WAITINGADM_hist_colour,total_tbs_hist_anomaly,total_tbs_hist_colour,vert_tbs_hist_anomaly,vert_tbs_hist_colour,pod_tbs_hist_anomaly,pod_tbs_hist_colour,overflow_hist_anomaly,overflow_hist_colour
0,2026-02-17 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,#107C10,no,#FFB900,no,#FFB900,no,#FFB900,no,#107C10
1,2026-02-17 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,#107C10,no,#FFB900,no,#FFB900,no,#FFB900,no,#FFB900
2,2026-02-17 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,#107C10,no,#FFB900,no,#FFB900,no,#FFB900,no,#FFB900
3,2026-02-18 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,#107C10,no,#FFB900,no,#FFB900,no,#FFB900,no,#107C10
4,2026-02-18 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,#FFB900,no,#FFB900,no,#FFB900,no,#FFB900,no,#107C10
5,2026-02-18 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,#FFB900,no,#107C10,no,#FFB900,no,#107C10,no,#107C10
6,2026-02-18 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,#FFB900,no,#FFB900,no,#FFB900,no,#107C10,no,#107C10
7,2026-02-18 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,#FFB900,no,#FFB900,no,#FFB900,no,#107C10,no,#107C10
8,2026-02-18 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,#FFB900,no,#107C10,no,#FFB900,no,#107C10,no,#107C10
9,2026-02-18 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,#FFB900,no,#107C10,no,#107C10,no,#107C10,no,#107C10


In [96]:
dropbox_app_key = os.environ.get("DROPBOX_APP_KEY")
dropbox_app_secret = os.environ.get("DROPBOX_APP_SECRET")
dropbox_refresh_token = os.environ.get("DROPBOX_REFRESH_TOKEN")

# exchange the authorization code for an access token:
token_url = "https://api.dropboxapi.com/oauth2/token"
params = {
    "grant_type": "refresh_token",
    "refresh_token": dropbox_refresh_token,
    "client_id": dropbox_app_key,
    "client_secret": dropbox_app_secret
}
r = requests.post(token_url, data=params)

dropbox_access_token = r.json()['access_token']

dbx = dropbox.Dropbox(dropbox_access_token)

upload(dbx, 'chronos_forecast.csv', '', '',
            'chronos_forecast.csv', overwrite=True)
upload(dbx, 'ED_Hourly_Forecasts_Anomalies_v1.0.csv', '', '',
            'ED_Hourly_Forecasts_Anomalies_v1.0.csv', overwrite=True)
upload(dbx, 'forecast_variable_effects.csv', '', '',
            'forecast_variable_effects.csv', overwrite=True)

uploaded as b'chronos_forecast.csv'
uploaded as b'ED_Hourly_Forecasts_Anomalies_v1.0.csv'
uploaded as b'forecast_variable_effects.csv'


FileMetadata(client_modified=datetime.datetime(2026, 2, 16, 20, 46, 58), content_hash='e67091084ccd8e1bdd5edfb96548e0e89d3c74f2d3983d49269226560633e641', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:oNSmVCFixyQAAAAAAABxQQ', is_downloadable=True, media_info=NOT_SET, name='forecast_variable_effects.csv', parent_shared_folder_id=NOT_SET, path_display='/forecast_variable_effects.csv', path_lower='/forecast_variable_effects.csv', preview_url=NOT_SET, property_groups=NOT_SET, rev='64b235e6c549b7a19c0a3', server_modified=datetime.datetime(2026, 2, 19, 1, 39), sharing_info=NOT_SET, size=2710, symlink_info=NOT_SET)